Syntax natürlicher Sprachen, WS 2023/24

# 07 - Aufgabenblatt


In [1]:
import nltk
from nltk import Tree
from nltk import DependencyGraph
from spacy import displacy
from itertools import chain

In [2]:
def _tree_labeled(self, i):
    node = self.get_by_address(i)
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word + '(' + rel + ')', [self._tree_labeled(dep) for dep in deps]) if deps else word + '(' + rel + ')'

def tree_labeled(self):
    node = self.root
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split(); line_list.pop(0); line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))
    return "\n".join([i for i in sent_list[0:]])

def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())
    deps = [x for x in deps if x]
    ex, word_list, arc_list = [], [], []
    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1; head = int(dep[2]); label = dep[3]
        if head>line:
            start = index; end = head-1; direction = "left"
        else:
            start = head-1; end = index; direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})
    ex.append({"words": word_list,"arcs": arc_list})
    return ex

def show_dep_trees(sent_nr,style=1):
    sent = transform_nr_conll(sent_nr)
    dg = DependencyGraph(sent)
    if style == 0 or style == 2:
        tree_labeled = dg.tree_labeled()
        tree_labeled.pretty_print(unicodelines=True)   
    if style == 1 or style == 2:
        ex = displacy_dep_input(sent)
        html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

## Aufgabe 1 - Analyse komplexer Satzkonstruktion

### a) Sehen Sie sich den folgenden englischen Satz an. Können Sie sich jeweils zwei unterschiedliche Interpretationen vorstellen?

- *Visiting relatives can be tiresome.*


In [ ]:
#

### b) Erweitern Sie die gegebene Grammatik, so dass linguistisch korrekte Syntaxanalysen beider Interpretationen ausgegeben werden.

##### Verwenden Sie `VBG` für *Verb, gerund or present participle* (Penn-Treebank) und `VP-INF` für infinite Satzkonstruktionen.

In [3]:
sent = 'visiting relatives can be tiresome'

In [ ]:
grammar = nltk.CFG.fromstring("""

    VBG -> 'visiting'
    N -> 'relatives'
    AUX -> 'can'
    COP -> 'be'
    ADJ -> 'tiresome'
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

### c) Geben Sie auch die entsprechenden Dependenzstrukturanalysen an.

In [ ]:
# Analyse 1

sent_nr = """
1 x 0 ROOT
"""

show_dep_trees(sent_nr)

In [ ]:
# Analyse 2

sent_nr = """
1 x 0 ROOT
"""

show_dep_trees(sent_nr)

## Aufgabe 2 - Analyse komplexer Satzkonstruktion

#### Schreiben Sie eine CFG, so dass für den folgende Satz eine korrekte Syntaxanalyse ausgegeben wird. Geben Sie auch eine entsprechende Dependenzanalyse an.

- *Als ich ankam, habe ich gesehen, dass sie bereits dagewesen sein mussten.*


In [4]:
sent = 'als ich ankam   habe ich gesehen   dass sie bereits dagewesen sein mussten'

#### Konstituentenanalyse:

In [ ]:
grammar = nltk.CFG.fromstring(
    """

    N -> 'ich' | 'alles' | 'sie'
    V -> 'ankam' | 'gesehen' | 'dagewesen'
    AUX -> | 'habe'| 'sein' | 'mussten'
    COMP -> 'als' | 'dass'
    ADV -> 'bereits'
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

#### Dependenzanalyse:

In [ ]:
sent_nr = """
1 als 
2 ich 
3 ankam 
4 habe 
5 ich 
6 gesehen 
7 dass 
8 sie 
9 bereits 
10 dagewesen 
11 sein 
12 mussten 
"""

show_dep_trees(sent_nr)